# Transformer

# Teoria
------

## Introdução

### RNNs clássicas para NLP

- Palavras são codificadas em vetores
- Cada novo estado é baseado no estado anterior
- Decodificação começa no estado final do codificador
![Rnns_classica](images/rnn_classica.png)

### Mecanismo de atenção

- ultima camada do codificador sobrecarregada com todos os textos
- atenção maior para as camadas
- maiores pesos para o contexto estado anterior

![Mecanismo de atenção](images/attention.png)

## Arquitetura 

Essa arquitetura tem como foco os mecanismos de atenção. Ao em vez de cada palavra receber a atenção, nos Transformers cada frase terá esse processo.

![arquitetura](images/transformer.png)
![arquitetura](images/self-attention.png)


## Scale-dot product

**Ideia principal:**
- 2 sequências (iguais no caso de self-attention), A e B
- calcular como cada elemento de A está relacionado a cada elmento de B
- depois recombinamos A de acordo com essa relação

**Matematicamente**, dot-product indica a similaridade entre dois vetores.

![scale_dot-product](images/scale_dot-product.png)

![scale_dot-product](images/scale_dot-product2.png)

## Look-ahead Mask

![look-ahead](images/look-ahead.png)

## Attention Layer

![attentio layer](images/attention-layer.png)

## Multi-head attention Layer

![multi-head](images/multi-head.png)

## Positional Encoding

![positional-encod](images/positional-encod.png)

## Feed-forward layers (camadas densas)

- composta de 2 transformações lineares

$$
FFN(x) = \max(0, x W_1 + b_1)W_2 + b_2
$$

## Residual connections:

- **Add & Norm:** não esquecer a informação da etapa anterior, ajudando a aprendizagem durante o *backpropagation*.
- **Last linear:** a saída do decodificador passa por uma camada densa de acordo com o tamanho do vocabulário e com a aplicação da função softmax, gerando probabilidades para cada palavra.

# Prática
----------

## Importação

In [1]:
import pandas as pd
import numpy as np

import math
import re
import time
import zipfile
import random

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

- Bases de dados: https://www.statmt.org/europarl/

In [2]:
file_path = '../../_IAExpert_private/7. Processamento de Linguagem Natural com Deep LEarning/pt-en'

with open(f"{file_path}/europarl-v7.pt-en.en", mode='r', encoding='utf-8') as f:
    europarl_en = f.read()

with open(f"{file_path}/europarl-v7.pt-en.pt", mode='r', encoding='utf-8') as f:
    europarl_pt = f.read()

In [3]:
europarl_en[:100]

'Resumption of the session\nI declare resumed the session of the European Parliament adjourned on Frid'

In [4]:
en = europarl_en.split('\n')
pt = europarl_pt.split('\n')

len(en), len(pt)

(1960408, 1960408)

In [5]:
i = random.randint(0, len(en)-1)
en[i], pt[i]

('For this reason, and with reservations as above, I have voted against the report.',
 'Por tal motivo, e pelas reservas supramencionadas, votei contra o relatório.')

### Limpeza dos dados

```python
corpus_en = europarl_en
corpus_en = re.sub(r"\.(?=[0-9][a-z][A-Z])", '.$$$', corpus_en)
corpus_en = re.sub(r".\$\$\$", '', corpus_en)
corpus_en = re.sub(r" +", ' ', corpus_en)
corpus_en = corpus_en.split('\n')

```

```python
corpus_pt = europarl_pt
corpus_pt = re.sub(r"\.(?=[0-9][a-z][A-Z])", '.$$$', corpus_pt)
corpus_pt = re.sub(r".\$\$\$", '', corpus_pt)
corpus_pt = re.sub(r" +", ' ', corpus_pt)
corpus_pt = corpus_pt.split('\n')
```

```python
with open("corpus.pkl", "wb") as f:
    pickle.dump([corpus_en, corpus_pt], f)
```

In [13]:
import pickle
with open("corpus.pkl", 'rb') as f:
    corpus_en, corpus_pt = pickle.load(f)

In [14]:
len(corpus_en), len(corpus_pt)

(1960408, 1960408)

### Tokenização

- texto para número

In [16]:
tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(corpus_en, target_vocab_size=2**13)

In [17]:
tokenizer_pt = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(corpus_pt, target_vocab_size=2**13)

In [18]:
tokenizer_en.vocab_size, tokenizer_pt.vocab_size

(8188, 8116)

In [19]:
vocab_size_en = tokenizer_en.vocab_size + 2
vocab_size_pt = tokenizer_pt.vocab_size + 2

In [ ]:
inputs_en = [[vocab_size_en - 2] + tokenizer_en.encode(sentence) + [vocab_size_en - 1] for sentence in corpus_en]

In [ ]:
inputs_pt = [[vocab_size_pt - 2] + tokenizer_pt.encode(sentence) + [vocab_size_pt - 1] for sentence in corpus_pt]